In [1]:
from elmoformanylangs import Embedder

/home/dutir923/irguest/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
class DecoderState_ensemble(object):
    """
    State of Decoder.
    """

    def __init__(self, states):
        """
        hidden: Tensor(num_layers, batch_size, hidden_size)
        """
        self.states=states
        self.mask=states[0].mask
        

    def __getattr__(self, name):
        return self.__dict__.get(name)

    def set_mask(self,):
        for state in self.states:
            state.mask=self.mask
    
    def get_batch_size(self):
        """
        get_batch_size
        """
        return self.states[0].get_batch_size()

    def size(self):
        """
        size
        """
        return self.states[0].size()

    def slice_select(self, stop):
        """
        slice_select
        """
        states=[]
        for state in self.states:
            states.append(state.slice_select(stop))
        return DecoderState_ensemble(states)

    def index_select(self, indices):
        """
        index_select
        """
        states=[]
        for state in self.states:
            states.append(state.index_select(indices))
        return DecoderState_ensemble(states)

    def mask_select(self, mask):
        """
        mask_select
        """
        states=[]
        for state in self.states:
            states.append(state.mask_select(mask))
        return DecoderState_ensemble(states)

    def inflate(self, times):
        """
        inflate
        """
        states=[]
        for state in self.states:
            states.append(state.inflate(times))
            
        return DecoderState_ensemble(states)





In [2]:
import torch
import torch.nn as nn
from source.utils.misc import Pack
import numpy as np

In [3]:
class ensemble(nn.Module):
    def __init__(self, models):
        super(ensemble, self).__init__()
        self.num=len(models)
        self.models=nn.ModuleList(models)
    def encode(self,inputs, enc_hidden):
        enc_outputs=Pack()
        states=[]
        elmo_embed=self.models[0].encoder.elmo_embedder.sents2elmo(inputs.raw_src)
        elmo_length = [x.shape[0] for x in elmo_embed]
        batch_size_1=len(elmo_length)
        max_l = max(elmo_length)
        size = (batch_size_1, max_l, 1024)
        tensor_1 = torch.zeros(size, dtype=torch.float)
        for i in range(batch_size_1):
            tensor_1[i][:elmo_length[i]] = torch.tensor(elmo_embed[i])
        elmo_embed=tensor_1.cuda()
        for i in range(self.num):
            _, state = self.models[i].encode_infe(inputs, elmo_embed, enc_hidden)
            states.append(state)
#         print(len(states))
        return enc_outputs, DecoderState_ensemble(states)
    
    def decode(self,input_var, dec_state):
#         print('-----')
        dec_state.set_mask()
        p=Pack()
        output_temp=[]
        attn_temp=[]
        dec_state_temp=[]
#         print(len(dec_state.states))
        for i in range(self.num):
#             state_=dec_state.states[i]
#             model_=self.models[i]
#             print(type(state_))
#             print(type(model_))
#             output, dec_state_0, attn = model_.decode(input_var, state_)
            output, dec_state_0, attn = self.models[i].decode(input_var, dec_state.states[i])
            output_temp.append(output)
            attn_temp.append(attn.attn)
            dec_state_temp.append(dec_state_0)
        
        output_temp=torch.stack(output_temp).mean(0)
        attn_temp=torch.stack(attn_temp).mean(0)
        p.add(attn=attn_temp)
        dec_state_temp=DecoderState_ensemble(dec_state_temp)
        return output_temp, dec_state_temp, p
            
            

In [4]:
file='./models/layer_2_64_embed_elmo/params.json'

In [5]:
import json

In [6]:
p_file=open(file,'r',encoding='utf-8')
pp=json.loads(p_file.read())
config=Pack(pp)

In [7]:
config.entity_file='./data/demo_neww_entity.txt'
# config.data_dir='./data/data500_pf/'

In [8]:
from source.inputters.corpus import KnowledgeCorpus, Entity_Corpus, Entity_Corpus_pos

In [9]:
config.gpu=1
device = config.gpu
torch.cuda.set_device(device)

In [10]:
corpus = Entity_Corpus_pos(data_dir=config.data_dir, data_prefix=config.data_prefix, entity_file=config.entity_file,
                             min_freq=config.min_freq, max_vocab_size=config.max_vocab_size)
corpus.load()

加载实体成功，size= 547394
Loading prepared vocab from ./data/data550_pos/model_50000.vocab.pt ...
Finish loading vocab , size: 50005
Finish loading pos vocab , size: 31
Loading prepared data from ./data/data550_pos/model_50000.data.pt ...
Number of examples: TRAIN-37226 VALID-2000


In [11]:
valid_iter = corpus.create_batches(
    15, "valid", shuffle=False, device=device)

In [12]:
from tqdm import tqdm
from source.models.Entity_seq2seq import Entity_Seq2Seq
from source.models.Entity_seq2seq_pos import Entity_Seq2Seq_pos
from source.models.Entity_seq2seq_pos_gru import Entity_Seq2Seq_pos_gru
from source.models.Entity_seq2seq_elmo import Entity_Seq2Seq_elmo
from source.models.Entity_seq2seq_elmo_gru import Entity_Seq2Seq_elmo_gru
from source.utils.generator import TopKGenerator
from source.utils.engine import evaluate, evaluate_generation

/home/dutir923/irguest/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:

file1='./models/layer_2_64_embed_elmo/params.json'
file2='./models/layer_2_64_embed_elmo_1/params.json'
file3='./models/layer_2_64_embed_elmo_2/params.json'
file4='./models/layer_2_64_embed_elmo_3/params.json'
file5='./models/layer_2_64_embed_elmo_4/params.json'

files=[file1,file2,file3,file4,file5]

def get_model(files):
    models=[]
    for f in tqdm(files):
        p_file=open(f,'r',encoding='utf-8')
        pp=json.loads(p_file.read())
        config=Pack(pp)
        config.batch_size=15
        if config.rnn_type=='GRU':
            
            model = model = Entity_Seq2Seq_elmo_gru(src_vocab_size=corpus.SRC.vocab_size,
                                   embed_size=config.embed_size, hidden_size=config.hidden_size,
                                   padding_idx=corpus.padding_idx,
                                   num_layers=config.num_layers, bidirectional=config.bidirectional,
                                   attn_mode=config.attn, with_bridge=config.with_bridge,
                                   dropout=config.dropout,
                                   use_gpu=config.use_gpu,
                                   pretrain_epoch=config.pretrain_epoch,
                                   batch_size=config.batch_size)
        else:
            model = model = Entity_Seq2Seq_elmo(src_vocab_size=corpus.SRC.vocab_size,
                                   embed_size=config.embed_size, hidden_size=config.hidden_size,
                                   padding_idx=corpus.padding_idx,
                                   num_layers=config.num_layers, bidirectional=config.bidirectional,
                                   attn_mode=config.attn, with_bridge=config.with_bridge,
                                   dropout=config.dropout,
                                   use_gpu=config.use_gpu,
                                   pretrain_epoch=config.pretrain_epoch,
                                   batch_size=config.batch_size)
        config.ckpt=config.save_dir+'/best.model'
        model.load(config.ckpt)
        models.append(model)
    return ensemble(models)

models=get_model(files)

  0%|          | 0/5 [00:00<?, ?it/s]2019-05-04 17:07:00,808 INFO: char embedding size: 6169
2019-05-04 17:07:01,318 INFO: word embedding size: 71222
2019-05-04 17:07:08,062 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (

Loaded model state from './models/layer_2_64_embed_elmo/best.model'


2019-05-04 17:07:09,712 INFO: char embedding size: 6169
2019-05-04 17:07:10,215 INFO: word embedding size: 71222
2019-05-04 17:07:14,325 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

Loaded model state from './models/layer_2_64_embed_elmo_1/best.model'


2019-05-04 17:07:15,496 INFO: char embedding size: 6169
2019-05-04 17:07:15,893 INFO: word embedding size: 71222
2019-05-04 17:07:20,260 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

Loaded model state from './models/layer_2_64_embed_elmo_2/best.model'


2019-05-04 17:07:23,059 INFO: char embedding size: 6169
2019-05-04 17:07:23,527 INFO: word embedding size: 71222
2019-05-04 17:07:28,385 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

Loaded model state from './models/layer_2_64_embed_elmo_3/best.model'


2019-05-04 17:07:29,398 INFO: char embedding size: 6169
2019-05-04 17:07:29,814 INFO: word embedding size: 71222
2019-05-04 17:07:34,316 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(71222, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(6169, 50, padding_idx=6166)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_feat

Loaded model state from './models/layer_2_64_embed_elmo_4/best.model'


In [14]:
config.max_dec_len=5
config.beam_size=5
generator = TopKGenerator(model=models,
                              src_field=corpus.SRC,
                              max_length=config.max_dec_len, ignore_unk=config.ignore_unk,
			      length_average=config.length_average, use_gpu=config.use_gpu, beam_size=config.beam_size)

In [15]:
evaluate_generation(generator, valid_iter, save_file=config.gen_file, verbos=True)

2019-05-01 22:33:43,300 INFO: 1 batches, avg len: 458.7
2019-05-01 22:33:46,635 INFO: 1 batches, avg len: 462.4
2019-05-01 22:33:49,192 INFO: 1 batches, avg len: 514.4
2019-05-01 22:33:52,422 INFO: 1 batches, avg len: 500.8
2019-05-01 22:33:55,455 INFO: 1 batches, avg len: 447.0
2019-05-01 22:33:58,367 INFO: 1 batches, avg len: 455.5
2019-05-01 22:34:00,764 INFO: 1 batches, avg len: 462.4
2019-05-01 22:34:03,158 INFO: 1 batches, avg len: 475.3
2019-05-01 22:34:05,538 INFO: 1 batches, avg len: 488.3
2019-05-01 22:34:07,923 INFO: 1 batches, avg len: 438.8
2019-05-01 22:34:10,333 INFO: 1 batches, avg len: 470.0
2019-05-01 22:34:12,691 INFO: 1 batches, avg len: 390.1
2019-05-01 22:34:14,989 INFO: 1 batches, avg len: 386.7
2019-05-01 22:34:17,258 INFO: 1 batches, avg len: 357.7
2019-05-01 22:34:19,607 INFO: 1 batches, avg len: 482.3
2019-05-01 22:34:21,931 INFO: 1 batches, avg len: 432.2
2019-05-01 22:34:24,364 INFO: 1 batches, avg len: 516.5
2019-05-01 22:34:27,089 INFO: 1 batches, avg len

Saved generation results to './test.result'
Avg_Len-2.055   F1-0.664   precision-0.709   recal-0.693   emo_precision-0.654
Target:   AVG_LEN-2.093   


In [15]:
corpus.reload(data_type='test')
test_iter = corpus.create_batches(
            15, "test", shuffle=False, device=device)

  7%|▋         | 211/3000 [00:00<00:01, 2109.95it/s]

finished data read


100%|██████████| 3000/3000 [00:01<00:00, 2954.72it/s]


Number of examples: TRAIN-37226 VALID-2000 TEST-3000


In [15]:
from source.inputters.dataset import Dataset, Entity_Dataset, Entity_Dataset_pos,Entity_Dataset_pos_sum

In [16]:
corpus.reload(data_type='test')
data=Entity_Dataset_pos(corpus.data["test"].data[:3000])
test_iter=data.create_batches(
            15,  shuffle=False, device=device)


  1%|          | 364/40000 [00:00<00:10, 3630.65it/s]

finished data read


100%|██████████| 40000/40000 [00:14<00:00, 2744.71it/s]


Number of examples: TRAIN-37226 VALID-2000 TEST-40000


In [16]:
config.gen_file='./result.txt545'
evaluate_generation(generator, test_iter, save_file=config.gen_file, verbos=True, for_test=True)

2019-05-04 17:10:34,732 INFO: 1 batches, avg len: 483.7
2019-05-04 17:10:37,697 INFO: 1 batches, avg len: 378.3
2019-05-04 17:10:40,677 INFO: 1 batches, avg len: 488.4
2019-05-04 17:10:43,530 INFO: 1 batches, avg len: 427.1
2019-05-04 17:10:46,449 INFO: 1 batches, avg len: 395.5
2019-05-04 17:10:48,902 INFO: 1 batches, avg len: 447.9
2019-05-04 17:10:51,378 INFO: 1 batches, avg len: 491.6
2019-05-04 17:10:53,936 INFO: 1 batches, avg len: 477.7
2019-05-04 17:10:56,736 INFO: 1 batches, avg len: 477.9
2019-05-04 17:10:59,961 INFO: 1 batches, avg len: 466.7
2019-05-04 17:11:03,120 INFO: 1 batches, avg len: 479.6
2019-05-04 17:11:05,945 INFO: 1 batches, avg len: 338.3
2019-05-04 17:11:08,716 INFO: 1 batches, avg len: 460.7
2019-05-04 17:11:11,731 INFO: 1 batches, avg len: 472.1
2019-05-04 17:11:14,934 INFO: 1 batches, avg len: 468.6
2019-05-04 17:11:17,454 INFO: 1 batches, avg len: 336.4
2019-05-04 17:11:20,497 INFO: 1 batches, avg len: 464.2
2019-05-04 17:11:23,343 INFO: 1 batches, avg len

2019-05-04 17:17:26,958 INFO: 1 batches, avg len: 432.3
2019-05-04 17:17:29,467 INFO: 1 batches, avg len: 348.9
2019-05-04 17:17:32,110 INFO: 1 batches, avg len: 468.3
2019-05-04 17:17:34,867 INFO: 1 batches, avg len: 473.3
2019-05-04 17:17:37,691 INFO: 1 batches, avg len: 478.3
2019-05-04 17:17:40,352 INFO: 1 batches, avg len: 485.0
2019-05-04 17:17:43,252 INFO: 1 batches, avg len: 395.9
2019-05-04 17:17:45,669 INFO: 1 batches, avg len: 367.8
2019-05-04 17:17:48,318 INFO: 1 batches, avg len: 465.3
2019-05-04 17:17:51,224 INFO: 1 batches, avg len: 436.2
2019-05-04 17:17:54,046 INFO: 1 batches, avg len: 474.5
2019-05-04 17:17:56,871 INFO: 1 batches, avg len: 439.0
2019-05-04 17:17:59,948 INFO: 1 batches, avg len: 498.0
2019-05-04 17:18:02,665 INFO: 1 batches, avg len: 447.9
2019-05-04 17:18:05,357 INFO: 1 batches, avg len: 491.3
2019-05-04 17:18:08,085 INFO: 1 batches, avg len: 430.4
2019-05-04 17:18:10,646 INFO: 1 batches, avg len: 425.2
2019-05-04 17:18:13,602 INFO: 1 batches, avg len

Avg_Len-2.452   F1-0.000   precision-0.000   recal-0.000   emo_precision-0.000
Target:   AVG_LEN-1.000   


Avg_Len-2.152   F1-0.646   precision-0.682   recal-0.688   emo_precision-0.649
Target:   AVG_LEN-2.093  
Avg_Len-2.168   F1-0.649   precision-0.680   recal-0.692   emo_precision-0.653
Target:   AVG_LEN-2.093   
Avg_Len-2.168   F1-0.649   precision-0.680   recal-0.692   emo_precision-0.653
Target:   AVG_LEN-2.093   
Avg_Len-2.055   F1-0.664   precision-0.709   recal-0.693   emo_precision-0.654
Target:   AVG_LEN-2.093   
Avg_Len-2.425   F1-0.640   precision-0.615   recal-0.729   emo_precision-0.693
Target:   AVG_LEN-2.093   
Avg_Len-1.730   F1-0.658   precision-0.744   recal-0.645   emo_precision-0.602
Target:   AVG_LEN-2.093

Avg_Len-2.006   F1-0.667   precision-0.711   recal-0.690   emo_precision-0.645
Target:   AVG_LEN-2.091  


Saved generation results to './test.result'
Avg_Len-2.055   F1-0.664   precision-0.709   recal-0.693   emo_precision-0.654
Target:   AVG_LEN-2.093  